In [1]:
import torch
import os 
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from transformers import TextStreamer, GenerationConfig

In [ ]:
access_tok = ''   # Hugging Face Access Token

In [2]:
default_path = os.getcwd()
data_path = os.path.join(default_path, '../../data')
model_path = os.path.join(default_path, '../../model')
config_path = os.path.join(default_path, '../../config')

In [3]:
model_name='davidkim205/komt-mistral-7b-v1'
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, low_cpu_mem_usage=True) # , device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
g_device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
model.to(g_device)

In [ ]:
g_device

In [ ]:
def gen(x):
    generation_config = GenerationConfig(
        temperature=0.8,
        top_p=0.95,
        max_new_tokens=512,
        # early_stopping=True,
        # do_sample=True,
    )
    q = f"[INST]{x} [/INST]"
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"\n\n### Response: "
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str

In [ ]:
user_input = '한국은행의 역할과 기능에 대해서 설명해보시요'

In [ ]:
gen(user_input)

#### PEFT 

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from transformers import TextStreamer, GenerationConfig

In [5]:
model='davidkim205/komt-mistral-7b-v1'
peft_model_name = 'davidkim205/komt-mistral-7b-v1-lora'
config = PeftConfig.from_pretrained(peft_model_name)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
config.base_model_name_or_path = model
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map="auto")
model = PeftModel.from_pretrained(model, peft_model_name)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
streamer = TextStreamer(tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
def gen(x):
    generation_config = GenerationConfig(
        temperature=0.8,
        top_p=0.95,
        max_new_tokens=512,
    )
    q = f"[INST]{x} [/INST]"
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        streamer=streamer,
    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"[/INST]"
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str

In [7]:
user_input = '블랙-숄즈 모델(Black-Scholes Model)이 무엇인지 설명해주세요.'

In [8]:
result = gen(user_input)

print('##########')
print(result)

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:394: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configurati

<s> [INST]블랙-숄즈 모델(Black-Scholes Model)이 무엇인지 설명해주세요. [/INST]블랙-숄즈 모델(Black-Scholes Model)은 주식 옵션의 가격을 예측하는 수학적 모델입니다. 이 모델은 주식의 가격이 시간이 지남에 따라 변동하는 것으로 가정하고, 이를 기반으로 옵션의 가격을 계산합니다. 이 모델은 1973년에 프랑스의 금융학자인 마티스 블랙과 프레드 숄즈에 의해 개발되었습니다. 

블랙-숄즈 모델은 주식의 가격이 시간이 지남에 따라 변동하는 것으로 가정하고, 이를 기반으로 옵션의 가격을 계산합니다. 이 모델은 주식의 가격이 시간이 지남에 따라 변동하는 것으로 가정하고, 이를 기반으로 옵션의 가격을 계산합니다. 

블랙-숄즈 모델은 주식의 가격이 시간이 지남에 따라 변동하는 것으로 가정하고, 이를 기반으로 옵션의 가격을 계산합니다. 이 모델은 주식의 가격이 시간이 지남에 따라 변동하는 것으로 가정하고, 이를 기반으로 옵션의 가격을 계산합니다. 

블랙-숄즈 모델은 주식의 가격이 시간이 지남에 따라 변동하는 것
##########
블랙-숄즈 모델(Black-Scholes Model)은 주식 옵션의 가격을 예측하는 수학적 모델입니다. 이 모델은 주식의 가격이 시간이 지남에 따라 변동하는 것으로 가정하고, 이를 기반으로 옵션의 가격을 계산합니다. 이 모델은 1973년에 프랑스의 금융학자인 마티스 블랙과 프레드 숄즈에 의해 개발되었습니다. 

블랙-숄즈 모델은 주식의 가격이 시간이 지남에 따라 변동하는 것으로 가정하고, 이를 기반으로 옵션의 가격을 계산합니다. 이 모델은 주식의 가격이 시간이 지남에 따라 변동하는 것으로 가정하고, 이를 기반으로 옵션의 가격을 계산합니다. 

블랙-숄즈 모델은 주식의 가격이 시간이 지남에 따라 변동하는 것으로 가정하고, 이를 기반으로 옵션의 가격을 계산합니다. 이 모델은 주식의 가격이 시간이 지남에 따라 변동하는 것으로 가정하고, 이를 기반으로 옵션의 가격을 계산합니다. 

블랙-숄즈 모델은 주식의 